### Nếu mở rộng từ **Groom Deformer bằng ML** sang **Motion & Skin (xương + da + cơ)** thì pipeline vẫn giữ ý tưởng chung (pose → deformation), nhưng phức tạp hơn vì thay vì chỉ “guides” ta phải xử lý cả **skin mesh, muscle volume, và motion sequence**. Dưới đây là phân tích workflow Houdini (SOP + TOP + ML nodes) 

---

## 1. Nguyên lý mở rộng từ Groom → Motion & Skin

- **Groom case**:
  - Input: Joint pose
  - Output: Guide deformation (nén PCA)
- **Skin/Motion case**:
  - Input: Joint pose / animation frame
  - Output:
    - Skin deformation (vertex delta so với bind pose)
    - Muscle simulation (quasi-static ground truth)
    - Motion dynamics (velocity/acceleration fields)
- Bản chất: vẫn là **mapping pose → deformation**, chỉ thay target từ “hair guides” sang “skin/muscle/mesh”.

---

## 2. Pipeline ML cho Motion & Skin

1. **Xác định Input Features (Pose Parameters)**
   - Joint rotations (Euler/quaternion)
   - Joint positions (world/local space)
   - Có thể thêm: velocity, acceleration của joint (nếu muốn motion-aware)
   - Normalize:
     - Rotation → quaternion (4D, tránh gimbal lock)
     - Position → relative to root (loại bỏ translation toàn thân)

2. **Sinh Pose Dataset (Pose Randomization)**
   - Dùng `ml pose generate SOP` như groom, mở rộng giới hạn khớp dựa trên motion capture hoặc animation library
   - Xuất ra nhiều pose + motion clip sample

3. **Ground-truth Deformation (Skin/Muscle Simulation)**
   - Với mỗi pose:
     - Dùng Muscle & Tissue solver trong Houdini (hoặc FEM) để tính skin deformation
     - Nếu cần motion: chạy simulation short sequence để lấy velocity field
   - Output:
     - Vertex positions/Δ (N×3)
     - Muscle tension map
     - Skin sliding data

4. **PCA Compression (Skin/Motion Deformation)**
   - Skin mesh có hàng chục nghìn vertex, cần PCA nén lại:
     - Tạo ma trận $M \in \mathbb{R}^{N \times (V \times 3)}$, với V = số vertex
     - Dùng PCA → latent space (K = 64–512)
   - Motion dynamics: có thể dùng PCA riêng trên velocity field hoặc Autoencoder 3D
   - Tách PCA theo body regions (torso, arm, leg, face) giúp model chính xác hơn

5. **Train ML Model (ML Train TOP)**
   - Input: Pose features (joint quaternions + pos)
   - Output: PCA coefficients (skin/muscle)
   - Model:
     - MLP: đủ cho skin deformation cơ bản
     - RNN/LSTM: nếu cần motion dynamics theo chuỗi
   - Loss:
     - MSE cho PCA coefficients
     - Có thể thêm “Laplacian smoothness” loss để tránh mesh rung

6. **Apply Model (ML Apply SOP)**
   - Với pose mới hoặc sequence mocap:
     - Dự đoán PCA coefficients
     - Reconstruct deformation (basis × coeff)
     - Áp dụng vào skin mesh → deform trực tiếp
   - Kết quả: Mesh có deformation gần giống muscle sim, nhưng realtime

---

## 3. Node Setup Houdini (so với Groom)

- **Groom Deformer**
  - `ML Example SOP` (features = joints, targets = guide deform)
  - `ML Train TOP` (regression → PCA coeffs)
  - `ML Apply SOP` (pose → guide deform)
- **Motion & Skin Deformer**
  - `ML Example SOP` (features = joints, targets = skin vertex Δ, muscle field)
  - `ML Train TOP` (regression → PCA coeffs cho skin/muscle)
  - `ML Apply SOP` (pose → skin deform)

---

## 4. Điểm cần lưu ý khi mở rộng

1. **Dung lượng dữ liệu**
   - Skin mesh >> groom guides
   - Nên nén PCA mạnh (64–128 component / vùng) hoặc dùng Autoencoder

2. **Chia vùng cơ thể**
   - Xử lý mesh theo cụm (torso, head, limbs)
   - Huấn luyện ML model riêng cho từng cụm

3. **Motion-awareness**
   - Nếu cần chuyển động mượt theo thời gian: input thêm velocity/acceleration của joints
   - Có thể train RNN/LSTM để tránh popping frame-to-frame

4. **Kết hợp Groom + Skin**
   - Sau khi có skin deformation ML → chạy groom deformer ML
   - Pipeline unified: Pose → Skin + Groom deformation

---

## 5. Ứng dụng trong công việc của bạn

- Trong game/film, thay vì chạy muscle sim nặng, bạn huấn luyện ML để thay thế:
  - Pose input từ animator → ML dự đoán skin deform
  - Groom deformer ML chạy song song
- Kết quả: **real-time preview**, phù hợp pipeline Houdini → Unreal Engine/UE5 (bake ML model thành runtime graph)

---

**Tóm tắt:**  
- ML Groom Deformer là case study nhỏ.  
- Mở rộng sang Motion & Skin tức là thay target từ “tóc” sang “mesh/cơ/da”, giữ nguyên workflow:  
  - Pose parameters → PCA → ML Train → ML Apply → Reconstruct deformation


### Trước khi mình ghép Groom vào, ta nên phân tích **Motion & Skin Deformer bằng Deep Learning (không chỉ ML truyền thống)** một cách chi tiết và chuyên sâu. Đây sẽ là nền tảng để sau đó bạn kết hợp Groom và có một pipeline “cơ – da – lông” hoàn chỉnh.

---

# 1. Vấn đề cơ bản khi dự đoán **Motion & Skin**

* **Input**: Pose hoặc motion (thường là joint rotations/quaternions, joint positions, velocity/acceleration).
* **Output**:

  * Skin deformation (mesh vertex Δ hoặc blendshape weights).
  * Muscle field (nếu mô phỏng cơ).
  * Motion dynamics (nếu theo sequence).

**Thách thức**: Mesh có hàng chục nghìn vertex → dữ liệu quá lớn, không thể trực tiếp train DL trên raw vertices.

👉 Giải pháp: **giảm chiều dữ liệu bằng PCA / Autoencoder / Graph-based embedding**.

---

# 2. Deep Learning Pipeline cho Motion & Skin

### 2.1. Biểu diễn Input Pose

* **Quaternions** cho rotation (ổn định, không gimbal lock).
* **Joint positions relative to root** (đảm bảo invariant theo translation).
* **Velocity / Acceleration** (nếu muốn capture motion dynamics).
* Normalize input để toàn bộ nằm trong \[-1, 1].

---

### 2.2. Biểu diễn Output Deformation

Có 3 hướng chính:

1. **PCA Latent Space** (truyền thống nhưng đơn giản):

   * Nén mesh deformation vào latent vector (64–512 dims).
   * Model DL chỉ cần dự đoán latent → reconstruct lại mesh.

2. **Autoencoder (AE / VAE)**:

   * Train một AE riêng cho mesh deformation (pose-driven).
   * Encoder nén deformation thành latent space.
   * Decoder reconstruct lại deformation.
   * Sau đó train model chính để mapping: pose → latent.

3. **Graph Neural Network (GNN)** (tiên tiến hơn):

   * Mesh = graph (vertex + edge).
   * Train GNN để dự đoán vertex deformation trực tiếp.
   * Lợi: giữ topology tự nhiên.
   * Khó: training nặng, dataset phải rất lớn.

---

### 2.3. Lựa chọn Kiến trúc Deep Learning

* **MLP (baseline)**:

  * Input: Pose vector (joints).
  * Output: Latent deformation vector (PCA coeffs / AE latent).
  * Ưu: Dễ train, nhanh inference.
  * Nhược: Khó capture motion temporal dynamics.

* **RNN / LSTM / GRU**:

  * Input: Pose sequence (frame-by-frame).
  * Output: Sequence latent deformation.
  * Ưu: Giữ continuity trong animation (không popping).
  * Nhược: Khó train dài hạn, inference kém real-time.

* **Transformer (Temporal Attention)**:

  * Input: Motion sequence (frames).
  * Output: Sequence deformation latent.
  * Ưu: Capture context xa (long-term motion).
  * Nhược: Training tốn GPU, cần nhiều data.

* **Graph Convolutional Network (GCN/GNN)**:

  * Input: Joint graph / Mesh graph.
  * Output: Vertex displacement.
  * Ưu: Giữ topology xương + mesh tự nhiên.
  * Nhược: Phức tạp, khó triển khai real-time trong game.

👉 Với film/game, pipeline hay dùng là **Pose (quaternion) → MLP → PCA latent → Mesh deformation**. Nếu mở rộng cho motion liên tục thì thêm **RNN hoặc Transformer** vào MLP.

---

# 3. Quy trình Training chuyên sâu

### 3.1. Data Preparation

* Sinh **pose randomization** (từ motion capture hoặc joint limits).
* Chạy muscle/skin simulation để lấy ground-truth mesh deform.
* Nén bằng PCA/Autoencoder.

### 3.2. Training Loss

* **Reconstruction Loss (MSE)**: so sánh mesh predicted vs mesh sim.
* **Smoothness Loss**: penalize khi mesh jitter frame-to-frame.
* **Regularization Loss**: tránh overfit.
* **Physics-informed Loss (tùy chọn)**: giữ volume, collision penalty.

### 3.3. Evaluation Metrics

* Vertex-to-vertex L2 distance.
* Normal consistency (surface smoothness).
* Temporal coherence (mesh không rung).

---

# 4. Workflow Node trong Houdini (Deep Learning oriented)

### 4.1. Data Creation

* SOP: `ml pose generate` → sinh pose dataset.
* TOP: chạy sim skin/muscle cho mỗi pose.
* SOP: xuất deformation (delta).

### 4.2. Latent Compression

* SOP: PCA (truyền thống) hoặc Python Node chạy Autoencoder offline.
* Xuất latent dataset.

### 4.3. Training

* TOP: `ML Train`

  * Input: joint pose features.
  * Target: PCA latent / AE latent.
  * Model: chọn MLP hoặc MLP+RNN.

### 4.4. Inference

* SOP: `ML Apply`

  * Input: new pose (hoặc sequence).
  * Output: latent deformation → reconstruct mesh skin.

---

# 5. Hướng tối ưu cho **Motion & Skin bằng DL**

1. **Tách vùng cơ thể**: train riêng torso, head, limbs → tăng accuracy.
2. **Temporal model**: dùng RNN/LSTM nếu motion continuity quan trọng.
3. **Hybrid PCA + Autoencoder**: PCA giảm chiều trước, AE refine để giữ detail.
4. **Export Runtime**: model sau training có thể convert thành ONNX/TensorRT để chạy real-time trong UE5.

---

# 6. So sánh: Groom vs Motion & Skin

| Groom ML Deformer                         | Motion & Skin DL Deformer                                          |
| ----------------------------------------- | ------------------------------------------------------------------ |
| Input = pose → Output = guide deformation | Input = pose (hoặc motion) → Output = skin/muscle mesh deformation |
| PCA 64–128 comp. đủ                       | PCA/AE cần 128–512 comp. cho skin                                  |
| MLP thường là đủ                          | Có thể cần RNN/Transformer cho motion continuity                   |
| Training nhanh, dataset nhỏ               | Dataset lớn (mô phỏng skin), training nặng hơn                     |
| Dùng để layer hair/fur đẹp hơn            | Dùng để thay thế muscle sim / skin sliding                         |

---

👉 Tóm tắt gắn gọn:

* Với **Motion & Skin**, deep learning nên đi theo hướng **Pose → PCA/AE latent → MLP/RNN → Mesh Deform**.
* Groom thì chỉ cần **MLP + PCA** là đủ.
* Khi kết hợp: bạn sẽ có pipeline **Pose → Skin Deform → Groom Deform**, cả hai đều chạy ML inference real-time thay vì simulation.



### Tuyệt — dưới đây là “bản thiết kế” chi tiết cho **workflow Deep Learning: Motion & Skin trong Houdini** và các **cách export sang Unreal Engine (UE)**. Mình chia thành: (A) pipeline sản xuất dữ liệu + huấn luyện trong Houdini, (B) suy luận & kiểm thử trong Houdini, (C) các con đường đưa sang UE (tùy yêu cầu realtime/độ linh hoạt), kèm checklist thực thi.

---

# A) Houdini: Pipeline tạo dữ liệu & huấn luyện DL cho Motion & Skin

```
[Anim Rig / Rest Rig]
        │
        ├─► (1) Pose Generation (SOP)  →  tập pose / motion clip
        │         • ml pose generate SOP
        │         • Giới hạn khớp / sampling theo Gaussian
        │
        ├─► (2) GT Deformation (TOPs)  →  ground-truth skin/muscle
        │         • Muscle/Tissue/FEM (quasi-static)
        │         • Xuất per-vertex Δ (so với bind), normal/tangent (tuỳ)
        │
        ├─► (3) Feature Builder (SOP) →  vector hoá input
        │         • Joint quaternions (khuyến nghị), pos rel-root
        │         • (tuỳ) velocity/accel cho motion-aware
        │         • Normalize & pack thành attributes
        │
        ├─► (4) Latent Compression     →  giảm chiều output
        │         • PCA (64–512)   hoặc   Autoencoder (AE/VAE)
        │         • Tách vùng (torso/head/arm/leg) để tăng fidelity
        │
        └─► (5) ML/DL Train (TOPs)     →  học pose→latent
                  • MLP (baseline), +RNN/Transformer nếu cần temporal
                  • Loss: MSE(recon) + temporal smooth + (tuỳ) Laplacian
                  • Xuất model + stats (mean/std), PCA basis/AE decoder
```

### (1) Pose Generation

* `ml pose generate SOP`: nhập skeleton, đặt joint limits, số lượng pose, seed; nếu có motion libraries → xen kẽ “random pose” và “pose từ clip”.
* Lưu cùng **joint name map** (để khớp với UE sau này).

### (2) Ground-truth Deformation (GT)

* TOP graph: mỗi pose chạy muscle/skin sim → xuất **per-vertex delta** (`@dP`) so với bind.
* Lưu thêm **mask/region id** cho chia vùng PCA/AE.

### (3) Feature Builder

* Tạo feature vector ổn định:

  * Rotation → **quaternion (x,y,z,w)** từng joint, theo local hoặc component space thống nhất.
  * Position → **relative-to-root**, đã scale-normalize (chiều cao nhân vật).
  * (Tuỳ) **dPose/dt** nếu bài toán temporal.
* Chuẩn hoá vào \[-1,1] hoặc z-score.

### (4) Latent Compression

* **PCA**:

  * Per-region PCA (vd: torso=128, arms=64/arm, legs=64/leg, head=128).
  * Lưu: `basis_[region].npy`, `mean_[region].npy`, `scale.json`.
* **AE/VAE** (nếu muốn giữ phi tuyến tốt hơn):

  * Train AE riêng trong TOPs (Python job) → `decoder_[region].onnx` + `encoder` (dùng trong training; runtime chỉ cần decoder nếu model chính xuất latent).

### (5) ML/DL Train

* **Input**: pose features (F).
* **Target**: latent (K) của từng region (ghép lại hoặc train multi-head).
* **Model**:

  * Real-time: **MLP** (2–4 hidden layers, 128–512 units).
  * Temporal: **MLP + GRU/LSTM** hoặc **Temporal Transformer** (window 8–32 frames).
* **Loss**:

  * `L = MSE(latent_pred, latent_gt)`
  * * (tuỳ) smoothness qua thời gian (TV loss)
  * * (tuỳ) Laplacian reg trên mesh khi reconstruct để hạn chế jitter.
* **Artifacts cần xuất**:

  * `pose2latent.onnx` (hoặc `.mlmodel` nếu dùng ML nodes của Houdini).
  * `pca_basis_*.npy` + `pca_mean_*.npy` **hoặc** `decoder_*.onnx` (AE).
  * `feature_stats.json` (mean/std của features, joint order).
  * `skeleton_map.json` (tên xương, thứ tự, không gian tọa độ).

---

# B) Houdini: Suy luận + kiểm thử nội bộ

```
[Pose (SOP)] → Normalize → pose2latent (ONNX/ML node)
      → (PCA) latent → basis×coeff + mean → per-vertex Δ   \
      → (AE)  latent → decoder(ONNX)      → per-vertex Δ    } → Apply to mesh → Render/QA
```

* **ML Apply SOP** (nếu dùng ML nodes Houdini) *hoặc* Python SOP gọi ONNX Runtime để chạy:

  * Lấy pose hiện tại → chuẩn hoá → **pose2latent**.
  * Reconstruct:

    * PCA: `dP = B·c + μ` (per-region rồi ghép).
    * AE: `dP = Decoder(c)`.
  * Áp vào mesh → so sánh GT vs Pred (L2, angle normal, temporal jitter).
* **QA nhanh**:

  * Heatmap `|dP_pred - dP_gt|`, đồ thị loss theo frame.
  * Kiểm tra các pose biên (extreme) có “nổ lưới”/self-intersection không.

---

# C) Export sang Unreal Engine (nhiều con đường, chọn theo nhu cầu)

## C.1) **PCA Morph Targets + Runtime Coefficients (khuyến nghị, gọn – realtime tốt)**

**Ý tưởng**: Export PCA basis thành **morph targets**; trong UE, một model nhỏ **dự đoán K hệ số** rồi **điều khiển morphs**.

**Chuỗi thực thi**

1. **Houdini → FBX Skeletal Mesh**: bind mesh + skeleton (trùng tên joints với UE).
2. **Houdini → Morph Targets**: với mỗi **PCA component** của từng region, bake thành 1 morph target (đặt tên quy ước):

   * `MT_Torso_PC_000 ... MT_Torso_PC_127`
   * `MT_ArmL_PC_000 ...`
   * (Giới hạn morph UE \~thực tế vài trăm; chia region để kiểm soát số lượng)
3. **Houdini → JSON stats**:

   * `feature_stats.json`, `skeleton_map.json`, `pca_mean_*.npy` (không cần basis ở UE vì basis đã “hóa thân” thành morphs).
4. **Model pose→coeff**:

   * Xuất `pose2latent.onnx`.
5. **UE Runtime**:

   * **Đọc pose xương** trong Anim Graph (Control Rig hoặc trực tiếp từ Pose Link).
   * Chuẩn hoá theo `feature_stats`.
   * **ONNX Runtime** (plugin) chạy `pose2latent.onnx` → nhận vector hệ số K cho từng region.
   * **Set Morph Target Weights** = hệ số (có thể scale/offset).
   * (Tuỳ) Low-pass filter nhỏ để mượt.

**Ưu**:

* Không phải update toàn bộ vertices bằng compute shader → rẻ.
* Ăn khớp pipeline anim/morph có sẵn của UE.
  **Nhược**:
* Số morph target nhiều → tăng memory & kích thước asset.
* Cần kỷ luật đặt tên/quản lý region.

---

## C.2) **UE ML Deformer (Plugin)**

**Ý tưởng**: Dùng hạ tầng ML Deformer của UE để train/infer ngay trong UE, nhưng **dataset** được “nấu” từ Houdini.

**Chuỗi thực thi (khái niệm)**

1. Từ Houdini export **neutral mesh**, **skeletal anim**, và **target deltas** theo từng pose/frame (có thể ở dạng morph frame hoặc point cache).
2. Trong UE ML Deformer, tạo asset **Training Data** bằng dữ liệu này (pose → target delta).
3. Train trong UE để có **runtime deformer** (NN + graph UE).

**Ưu**:

* Trọn hệ sinh thái UE, trình bày tốt, ít custom code.
  **Nhược**:
* Phụ thuộc phiên bản UE & plugin; data ingestion cần đúng định dạng; ít kiểm soát hơn so với PCA-morph/ONNX tự do.

---

## C.3) **ONNX Runtime + Compute Shader (vertex Δ trực tiếp)**

**Ý tưởng**: Không dùng morphs. Model xuất **per-vertex delta** (K lớn). Dùng **Compute Shader** cập nhật buffer vertex mỗi frame.

**Chuỗi thực thi**

1. Export `pose2latent.onnx` và **`decoder.onnx`** (AE) hoặc `pca_basis_*.npy` nếu vẫn PCA (khi đó compute shader chỉ làm `B·c+μ`).
2. UE C++ plugin:

   * Lấy xương → build feature → ONNX infer (pose→latent).
   * **AE**: ONNX infer tiếp (latent→Δ) **hoặc**
     **PCA**: compute shader nhân ma trận `B·c` trên GPU (rất nhanh) + cộng mean.
   * Ghi Δ vào **deform buffer** rồi áp lên mesh.

**Ưu**:

* Tự do nhất, không giới hạn morph.
  **Nhược**:
* Cần kỹ năng C++/RHI/compute shader; quản lý đồng bộ buffer, skin pass.

---

## C.4) **Vertex Animation Texture (VAT) / Point Cache (baked)**

* Dùng cho **phi tương tác** (cinematic/previz).
* Xuất dãy Δ theo frame → bake vào texture → UE đọc texture animate.
* Không phản hồi pose realtime → phù hợp “shot-based”.

---

# D) Chuẩn hoá dữ liệu & đặt tên (để 2 chiều HDA ↔ UE “ăn rơ”)

* **Skeleton**: joint names & hierarchy **y hệt** giữa Houdini và UE.
* **Không gian tọa độ**: chọn 1 kiểu (component/local), thống nhất trong `skeleton_map.json`.
* **Feature Stats**: `mean/std` để normalize/inverse normalize trong UE (đảm bảo đầu vào model đúng scale).
* **Morph Targets** (C.1):

  * Quy ước: `MT_<Region>_PC_<idx>`; lưu kèm `region_ranges.json` để biết mapping morph ↔ latent index.
* **PCA/AE Artifacts**:

  * PCA: `pca_mean_*.npy`, (nếu không bake morph) `pca_basis_*.npy`.
  * AE: `decoder_*.onnx` (runtime), `encoder` chỉ dùng train.

---

# E) Quy trình kiểm thử khi sang UE

1. **Unit test** vài pose chuẩn (A-pose, T-pose, extreme bends): so sánh mesh UE vs Houdini (L2/vertex, angle normal).
2. **Stress test** motion nhanh: kiểm tra jitter, “nổ lưới”, trễ khung.
3. **Hiệu năng**:

   * Morph route: đo CPU cost đặt morphs + skinned mesh pass.
   * Compute route: đo GPU time của compute + skin.
4. **Fallback**: nếu model không tải được → rơi về skinning truyền thống.

---

# F) Lộ trình gợi ý (thực dụng, dễ triển khai)

* **Giai đoạn 1 (baseline)**: PCA theo vùng + **MLP** → **C.1 (Morph + ONNX)**. Nhanh, đẹp, dễ debug.
* **Giai đoạn 2 (nâng cấp temporal)**: thêm **GRU** hoặc Temporal Transformer (cửa số 16–32 frames), vẫn giữ PCA-morph.
* **Giai đoạn 3 (tối đa hóa chi tiết)**: thay PCA bằng **AE decoder** + compute shader (C.3) để vượt giới hạn morph; hoặc chuyển hẳn sang **UE ML Deformer** nếu pipeline của team thiên UE.

---

## Mẫu pseudo-export (tham khảo tên file)

```
/export/
  skeletal_mesh.fbx
  feature_stats.json         # mean/std, order joints, spaces
  skeleton_map.json          # name remap HDA↔UE
  regions.json               # torso/arms/legs/head vertex groups
  # PCA route:
  pca_mean_torso.npy
  pca_basis_torso.npy        # (nếu không bake thành morph)
  ...
  # AE route:
  decoder_torso.onnx
  decoder_armL.onnx
  ...
  pose2latent.onnx
```

-

### Tuyệt — mình sẽ làm hai phần:

1. **Checklist Houdini (SOP/TOP) cho pipeline PCA Morph + ONNX**
2. **Sơ đồ Anim Blueprint trong UE để chạy inference + apply morph target**

---

# 1. Checklist Houdini – PCA Morph + ONNX

### (A) Chuẩn bị dữ liệu

* **SOP Level**

  * Import Rigged Mesh (FBX/Character HDA).
  * `ml pose generate SOP` → tạo bộ pose.
  * Lưu `skeleton_map.json` (order joint, joint name map UE).

* **TOP Level**

  * `ROP Fetch` → chạy muscle/skin sim cho từng pose.
  * Xuất **per-vertex Δ** (so với bind pose).

---

### (B) Feature Vector (Input)

* **Python SOP / Wrangle SOP**:

  * Extract joint rotations → quaternion.
  * Normalize root transform.
  * Xuất `feature.npy`.
* Ghi `feature_stats.json` (mean/std).

---

### (C) Latent Compression (Output)

* **Python Processor (TOP)**:

  * Run PCA per region (torso/arms/legs/head).
  * Save:

    * `pca_mean_region.npy`
    * `pca_basis_region.npy`
  * Rebuild Δ = basis·coeff + mean → QA.

* **Morph Target Baking (SOP)**:

  * Với mỗi PCA component → tạo mesh morphed.
  * Export ra **FBX Morph Targets** (naming convention: `MT_<Region>_PC_<Idx>`).

---

### (D) ML Training

* **ML Train TOP**:

  * Input = `features.npy`, Target = PCA coeffs.
  * Model = MLP (2–3 hidden layers, 256–512 units).
  * Loss = MSE.
  * Output = `pose2latent.onnx`.

---

### (E) Xuất package cho UE

```
/export/
  skeletal_mesh_with_morphs.fbx
  pose2latent.onnx
  feature_stats.json
  skeleton_map.json
  region_ranges.json   # mapping latent index -> morph target
```

---

# 2. UE – Anim Blueprint (PCA Morph + ONNX)

### **Anim Graph Flow**

```
[Input Pose from Control Rig / AnimBP]  
        │
        ▼
(Node) Extract Local Rotations
        │
        ▼
Normalize (use feature_stats.json)
        │
        ▼
ONNX Inference (pose2latent.onnx) → Latent Vector (coeffs)
        │
        ├─► Split by Region
        │       ├─ torso coeffs → set morph targets MT_Torso_PC_000...  
        │       ├─ armL coeffs → MT_ArmL_PC_000...  
        │       └─ etc.
        │
        ▼
Apply Morph Targets → Mesh
```

### **Implementation Notes**

* **ONNX Runtime plugin** (Epic/Marketplace).
* Blueprint function:

  * `GetBoneTransform` (for skeleton input).
  * Custom `NormalizeFeatures` (C++/BP).
  * `RunONNXModel` → outputs array coeffs.
  * `SetMorphTarget` in loop.
* (Tuỳ chọn) Filter coeffs (Exponential Smoothing) để mượt.

---

👉 Như vậy bạn có một pipeline khép kín:

* Houdini train + PCA → morph targets.
* Xuất ONNX model dự đoán coeff.
* UE AnimBP apply coeff vào morph.

--

### Tốt 👍, mình sẽ giải thích chi tiết từng khối trong sơ đồ:

---

# 🟦 Houdini (Data Prep + Training)

### **Rigged Mesh (FBX)**

* Nhập nhân vật đã rig từ Maya/Blender/UE.
* Đây là mesh + skeleton chuẩn để làm ground-truth.

### **Pose Generation (ml pose generate SOP)**

* Sinh tập pose đa dạng (rotation các joints).
* Đảm bảo bao phủ motion space → tránh overfit.

### **Muscle/Skin Simulation (TOPs)**

* Chạy muscle/skin sim (FEM hoặc Vellum).
* Kết quả: mesh deform chính xác theo pose.

### **Δ Vertex Cache**

* Tính delta mesh (so với bind pose).
* Đây là target ground-truth để train ML.

### **Feature Extract (Joint Rotations)**

* Lấy quaternion/rotation matrix cho mỗi joint.
* Normalize root để pose invariant.
* Xuất thành feature vector.

### **PCA Compression (per Region)**

* Nén delta mesh thành latent coeffs bằng PCA.
* Chia theo region (torso, head, arms, legs).
* Giảm từ hàng nghìn vertex → vài chục coeff.

### **Morph Target Baking (FBX Export)**

* Với mỗi PCA component → tạo một morph target.
* Xuất FBX kèm các morph target đó.

### **ML Train TOP (MLP → ONNX)**

* Input = joint rotations (feature).
* Output = PCA coeffs.
* Train MLP (2–3 hidden layers).
* Xuất model `pose2latent.onnx`.

### **Export Package**

Chứa tất cả file UE cần:

* skeletal\_mesh\_with\_morphs.fbx (mesh + morph target).
* pose2latent.onnx (model dự đoán).
* feature\_stats.json (mean/std).
* skeleton\_map.json (mapping joint order).

---

# 🟦 Unreal Engine (Inference + Application)

### **AnimBP – Extract Joint Rotations**

* Trong Animation Blueprint → lấy local bone rotation.
* Ánh xạ joint name theo `skeleton_map.json`.

### **Normalize Features**

* Chuẩn hóa theo mean/std từ Houdini (`feature_stats.json`).

### **ONNX Runtime Inference (pose2latent.onnx)**

* Chạy model → output latent vector (PCA coeffs).

### **Split Latent by Region**

* Cắt vector theo từng region.
* Ví dụ: 0–31 = torso, 32–63 = head, v.v.

### **Apply Morph Targets (SetMorphTarget)**

* Gán coeff vào morph target đã export (FBX).
* Có thể smoothing (Exponential Filter) để giảm jitter.

### **Final Skinned Mesh with Deformation**

* Nhân vật deform realtime → không cần muscle/skin sim.
* Ready cho game/film pipeline.

---

👉 Như vậy pipeline này:

* Houdini lo phần **Data Prep + PCA + Training**.
* UE lo phần **Inference + Runtime Deformation**.
* Ưu điểm: real-time, mô phỏng da cơ giống thật nhưng nhẹ.


### Rất tốt — mình sẽ phân tích **chuyên sâu, thực tế** các chiến lược tối ưu hoá để chạy inference Motion\&Skin realtime trong **Unreal Engine** cho nhiều nhân vật (crowd / AI characters). Mình chia nội dung thành: ý tưởng chính, các phương án (caching, GPU inference, compute shader reconstruction, LOD & batching), chi tiết triển khai (ONNX → TensorRT, lưu trữ dữ liệu, data layout), ví dụ pseudo-code / shader, và checklist đo đếm & profiling. Mình giữ mọi hướng dẫn cụ thể, có thể triển khai ngay.

---

# Tóm tắt nhanh (tl;dr)

* **Precompute & cache** latent vectors cho animation clips/poses khi có thể → tiết kiệm inference cho crowd.
* **Chạy inference trên GPU** (ONNXRuntime+CUDA hoặc TensorRT) với FP16/INT8 để tăng throughput.
* **Reconstruct deltas trên GPU** bằng compute shader (ma trận PCA × coeffs) thay vì CPU → rất nhanh cho nhiều instance.
* **LOD / region split / morph-count limit**: giảm kích thước latent hoặc chuyển sang VAT ở LOD thấp.
* **Packing & batching**: pack latent vào textures/SSBO để đọc nhiều instance bằng 1 draw/dispatch.
* **Quantize & prune** model để giảm memory và tăng perf.

---

# 1) Chiến lược chính — Khi nào tính trước (cache) / khi nào infer realtime

1. **Cache offline (nên ưu tiên)**

   * Nếu nhân vật dùng animation clip cố định (cinematic, locomotion cycles), **precompute latent cho mỗi frame** của clip. Lưu dưới dạng bảng (frame → latent vector).
   * Khi runtime có nhiều instance dùng cùng clip, chỉ cần **index và đọc latent** theo frame, không chạy inference.
   * Lưu ở dạng texture (latent texture) hoặc SSBO cho truy cập GPU.

2. **Realtime inference (khi cần tính pose động, random)**

   * Dùng ONNX/TensorRT để infer pose→latent realtime.
   * Chỉ infer cho “representative agents” hoặc cho agents có hành vi khác biệt; các agent khác tham chiếu kết quả (share) nếu cùng pose/clip.
   * Batch nhiều inference cùng lúc để tận dụng GPU throughput.

**Quy tắc:** nếu animation/pose có thể precompute → hãy precompute. Realtime inference nên dành cho trường hợp không thể dự đoán trước (AI-driven unique poses).

---

# 2) GPU inference: ONNX → TensorRT / ONNXRuntime (CUDA) — chọn gì?

* **ONNXRuntime (CUDA Execution Provider)**

  * Dễ tích hợp vào Unreal thông qua plugin ONNXRuntime.
  * Hỗ trợ FP32/FP16, multi-threading, có plugin Android/DirectML.
  * Tốt cho dev cycle nhanh.

* **TensorRT** (nếu target NVIDIA PC/Server)

  * Tối ưu tốt hơn cho throughput và latency, hỗ trợ FP16/INT8, layer fusion, kernel autotune.
  * Workflow: Export ONNX → build TRT engine (offline hoặc at startup) → load engine in UE C++ via TensorRT runtime.
  * **INT8** cần calibration dataset để giữ accuracy.

**Recommendation:** Cho game desktop trên NVIDIA — **TensorRT** cho production throughput lớn; cho đa nền tảng hoặc dev nhanh — **ONNXRuntime(CUDA/DirectML)**.

---

# 3) Sử dụng Mixed Precision / Quantization

* **FP16 (half)**: thường giảm mem ×2 và tăng throughput với ít mất mát accuracy. Nên bật nếu model hỗ trợ.
* **INT8**: giảm hơn nữa nhưng cần calibration và kiểm tra lỗi tái tạo latent. Dùng cho rất large scale crowd inference.

**Lưu ý**: kiểm tra MSE tăng khi quantize; nếu mất quá nhiều detail, dùng FP16.

---

# 4) Tối ưu batch & scheduling inference

* **Batch many inputs per infer call**: inference throughput trên GPU tốt nhất khi xử lý batch (ví dụ batch = 32/64).

* **Group agents by feature signature** (same skeleton order, same normalization): batch those together.

* **Inference pipeline**:

  1. Collect poses for a tick into batches (CPU)
  2. Copy batch to GPU input buffer (staging)
  3. Run model (ONNX/TensorRT) → get batch of latent vectors
  4. Store latent in GPU SSBO / texture for reconstruction pass

* **Asynchronous execution**: dispatch GPU inference/compute in background GPU queues, but coordinate frame sync so results available for rendering. (Implement via RHI command lists / async compute in UE C++.)

---

# 5) Reconstruction on GPU — ma trận PCA × coeffs (compute shader)

Thay vì tính `dP = B · c + μ` trên CPU cho từng vertex, chạy trên GPU. Lợi ích lớn khi có nhiều instances.

## Lưu trữ dữ liệu

* **Basis matrix B (V × K)**: lưu dưới dạng **texture 2D** (or structured buffer / SSBO) — mỗi hàng là một vertex; mỗi cột là một component.
* **Coefficients c (K)**: per-instance vector — lưu trong SSBO hoặc latent texture (1 texel row per instance).
* **Mean μ (V)**: per-vertex mean vector — lưu trong texture/SSBO.

## Compute shader (pseudo HLSL)

```hlsl
// Input:
// - RWTexture2D<float3> OutPosDelta;     // per-vertex output delta
// - Texture2D<float> BasisTex;           // dimensions (K, V) or (V, K) depending layout
// - StructuredBuffer<float> Coeffs;      // coefficients per instance (K) or big array for many instances
// - Texture2D<float3> MeanTex;           // mean per vertex

[numthreads(64,1,1)]
void CSMain(uint3 dispatchThreadID : SV_DispatchThreadID)
{
    uint vertexIdx = dispatchThreadID.x + dispatchThreadID.y*...; // depends dispatch
    float3 delta = float3(0,0,0);
    // For performance: unroll small K, or read blocks
    for (uint k=0;k<K;++k) {
        float basis_x = BasisTex.Load(int3(k, vertexIdx, 0)); // basis value for vertexIdx, component k => scalar
        // base stored as 3 channels if basis stores xyz per component; adjust accordingly
        delta += basis_x * Coeffs[k];
    }
    delta += MeanTex.Load(int3(vertexIdx,0,0));
    OutPosDelta[vertexIdx] = delta;
}
```

* Dispatch once per region per instance or dispatch multi-instance with additional indexing dimension.
* For many instances, you can run shader to handle `vertexIdx, instanceIdx` and write into per-instance vertex buffer or into GPU morph target buffer.

## Data layout tips

* Store `Basis` as **K textures each containing vec3 per vertex** so shader reads contiguous texel per k. Or pack K into RGBA texels to reduce fetches.
* Use **FP16 textures** for basis & mean to reduce memory and bandwidth.
* Coeffs for many instances can be a 2D texture: width=K, height=#instances. Sampling is cheap.

---

# 6) Caching latent vectors for crowd

### Approach A — Per-frame per-clip cache (best when clips known)

* For each clip, precompute latent per frame:

  * `latent_table[clipId][frame] = K-vector`
* Store as 2D texture: width=K, height=framesTotal or height=instances if shared.
* Runtime:

  * For agent using clip, compute frame index → sample latent texel row → use for reconstruction.

**Memory estimate example**

* K = 128, FP16(2 bytes) per component = 256 bytes / frame.
* Clip length = 300 frames → \~75 KB per clip.
* 100 clips → 7.5 MB. (very cheap)

### Approach B — Per-pose discretization (quantize the pose space)

* Discretize pose space (hash / kmeans clustering) → precompute latent for cluster centers.
* At runtime, find nearest cluster center for an agent → reuse cached latent.
* Good for large, varied animation but with redundancy.

### Approach C — Hybrid: cache for LOD0; infer for LOD0 agents only; LOD1/2 use cheaper VAT / baked anim.

* For far LODs, bake VAT or use normalized skinning only.

---

# 7) LOD, morph count & memory

* **Morph target limit**: UE supports many morphs but each added morph increases CPU cost when set via Blueprints (depends). GPU morphing via vertex shader is preferable for hundreds of morph targets.
* **Region split**: split body into regions with own K (torso K=128, arms K=64...). This keeps K per region manageable.
* **LOD mapping**:

  * LOD0: full PCA or AE decode via compute shader.
  * LOD1: fewer PCA components or fewer morph targets.
  * LOD2+: VAT or bake vertex animations.

---

# 8) Packing & transferring data efficiently to GPU

* **Pack coeffs into 2D texture**: sample with texelFetch to avoid filtering.
* **Use SSBO (StructuredBuffer) if on PC**: more flexible and faster for random access.
* **Avoid per-vertex CPU uploads**: all reconstruction should happen on GPU, CPU only submits parameters (coeff index / frame index).
* **Use persistent mapped buffers** for updates per frame to minimize copies.

---

# 9) Integrating into UE: practical pipeline

1. **Offline**

   * Export pose2latent.onnx (FP16 or INT8 with calibration).
   * Precompute & export PCA basis textures, mean textures (FP16).
   * Bake per-clip latent textures if possible.

2. **Runtime C++ plugin**

   * Initialize ONNX/TensorRT engine.
   * For each tick:

     * Gather batch of poses → run inference (or sample cache).
     * Upload batch latents into GPU buffer/texture.
     * Dispatch compute shader to reconstruct per-instance vertex deltas.
     * Use a custom vertex factory or material to read deltas and add to skinned positions.

3. **Rendering**

   * Vertex shader: skinning + add `delta` from reconstructed buffer or use morph target system (if morph route).
   * Ensure synchronization (Render thread vs RHI command list).

---

# 10) Example pseudo-workflow (UE C++ sketch)

```cpp
// 1) Collect poses for batch, fill input tensor (batchSize, featureDim)
// 2) if (useCache) sample latentTex[clipId][frameIdx] -> GPUUploadCoeffs(batchCoeffs)
//    else run ONNX/TensorRT inference -> GPUUploadCoeffs(batchCoeffs)
// 3) Dispatch compute shader:
//    - binds BasisSSBO, MeanTex, CoeffTex, OutDeltaBuffer
//    - Dispatch size = ceil(V / THREADS)
// 4) In vertex shader: fetch OutDeltaBuffer[vertexIdx + instanceOffset] -> add to skinned position
```

---

# 11) Profiling & metrics — cái gì cần đo

* **Latency per frame**: inference time (ms) + reconstruction compute (ms).
* **Throughput**: agents processed / ms (or fps for N agents).
* **Memory**: basis textures size, latent textures, ONNX engine memory.
* **Bandwidth**: texture fetches in compute shader (bytes read per frame).
* **Accuracy**: vertex L2 error vs Houdini ground-truth; temporal jitter metric.

---

# 12) Checklist triển khai (bước-bước)

1. **Prototype**: CPU-only ONNX runtime → PCA reconstruct on CPU → verify quality.
2. **Export ONNX FP32 → test in UE ONNX plugin**.
3. **Measure perf**; enable FP16 conversion → retest accuracy.
4. **Implement GPU reconstruction** (compute shader) reading coeffs from texture.
5. **Batching**: group agents per-animation to process many coeffs per infer call.
6. **Add cache layer**: per-clip per-frame latent textures for heavily reused clips.
7. **Implement LOD**: fallback to VAT or baked skin for far LOD.
8. **Optimize I/O**: use FP16 textures, pack channels, persistent mapped buffers.
9. **If target NVIDIA large crowd**: build TensorRT engine (FP16/INT8) offline and integrate.
10. **Profile** and adjust K components / model size / batch size.

---

# 13) Ví dụ con số để ước lượng (quick back-of-the-envelope)

* K = 128, FP16 → 256 bytes / latent.
* 1000 agents all unique /frame and no cache → 256 KB per frame to fetch latents (not large). But inference cost >> memory; so caching many agents is beneficial.
* Basis texture: V = 50k vertices, K=128 → 50k×128×2 bytes ≈ 12.5 MB (FP16) per region — feasible on modern GPU if split per region and reused.

---

# 14) Các rủi ro & lưu ý thực tế

* **Memory bandwidth**: reconstructing large meshes for many instances requires careful texture/SSBO layout to avoid bandwidth bottleneck.
* **Synchronization**: must manage GPU fences for inference → reconstruction → rendering.
* **Compatibility**: TensorRT only for NVIDIA; for other GPUs dùng ONNXRuntime+DirectML or Metal Performance Shaders on Apple.
* **Precision vs Quality**: aggressive quantization (INT8) có thể degrade subtle surface detail — test kỹ.

---

# Kết luận & gợi ý bắt tay làm ngay

1. **Bắt đầu bằng cache-based route**: precompute latents per clip, integrate GPU reconstruct shader — sẽ cho lợi ích lớn ngay.
2. **Tăng dần**: sau khi GPU reconstruct sẵn sàng, thêm ONNXRuntime batching; cuối cùng build TensorRT engine cho môi trường NVIDIA nếu cần throughput lớn.
3. **SPLIT vùng & LOD**: chia regions, dùng AE/PCA hợp lý, LOD fallback cho crowd.
